In [2]:
# Gloabl variables and import declarations
import StringIO
import argparse
from bs4 import BeautifulSoup
# Parser declarations
parser = argparse.ArgumentParser()
parser.add_argument('book_name', nargs='+', help ='the file name of the book to be converted') 
parser.add_argument('TOC_name',nargs = '+', help = 'the file name of the TOC to be generated') 
parser.add_argument('NCX_name',nargs = '+', help = 'the file name of the NCX to be generated') 
args = parser.parse_args()
the_book = open(str(args.book_name), "w+")
soup = BeautifulSoup(the_book)
print(soup.prettify())
finalToc = open(str(args.TOC_name), "w")
finalNcx = open(str(args.NCX_name), "w")
tempToc = StringIO.StringIO()
tempNcx = StringIO.StringIO()


SyntaxError: invalid syntax (<ipython-input-2-a0bd2147c20f>, line 12)

In [ ]:
print(args.book_name) #I just like being reminded of what you're converting

In [ ]:
#TOC and NCX templates
TOCTEMPLATE = BeautifulSoup("<html><head><title>Table Of Contents</title><link rel='stylesheet' type='text/css' href='BLStyleSheet.css'></head><body><ul></ul></body</html>")
NCXTEMPLATE = BeautifulSoup("""
<?xml version='1.0' encoding='UTF-8'?>
<!DOCTYPE ncx PUBLIC '-//NISO/DTD ncx 2005-1/EN' 'http://www.daisy.org/z3986/2005/ncx-2005-1.dtd'>
<ncx xmlsn='http://www.daisy.org z3986/2005/ncx/' version='2005-1' xml:lang='en-US'>
<head>
<meta content='uid' name='dtb:uid'/>
<meta content='1' name='dtb:depth'/>
<meta content='0' name='dtb:totalPageCount'/>
<meta content='0' name='dtb:maxPageNumber'/>
</head>
<docTitle><text></text></docTitle>
<docAuthor><text></text></docAuthor>
<navMap>
</navMap>
</ncx>""", "xml")

In [ ]:
# This is all one loop because weird artifacts and odd problems crop up if you do two loops. 
NCXTEMPLATE.docTitle.string = soup.title.string #puts the title in
NCXTEMPLATE.docAuthor.string = soup.h3.string #puts the author in. I don't like this method, but it works. 
for tag in reversed(soup.find_all("a", attrs={"id":True})):
    #tempId = userBookName + "#" + (tag["id"])
    TOCTEMPLATE.body.ul.insert(0,TOCTEMPLATE.new_tag("a"))
    TOCTEMPLATE.a["href"] = userBookName + '#' + (tag["id"])
    TOCTEMPLATE.a.string = tag.string
    NCXTEMPLATE.navMap.insert(0, NCXTEMPLATE.new_tag("navpoint", id = tag["id"], playorder = 0))
    NCXTEMPLATE.navMap.navpoint.insert(2,NCXTEMPLATE.new_tag("content", src = userBookName + '#' + (tag["id"])))
    NCXTEMPLATE.navMap.navpoint.insert(0, NCXTEMPLATE.new_tag("navLabel"))
    NCXTEMPLATE.navLabel.string = tag.string
    NCXTEMPLATE.navLabel.string.wrap(NCXTEMPLATE.new_tag("text"))
    

for tag in TOCTEMPLATE.find_all("a"): #this wraps the tags with <li> tags.
        tag.wrap(TOCTEMPLATE.new_tag("li")) 

In [ ]:
# This is the counter I use to increment the playorder value. 
counter = 3 
#and this finishes off the NCX by incrementing the playorder value
for tag in NCXTEMPLATE.find_all("navpoint"): 
    tag["playorder"] = counter
    counter = counter + 1   


In [ ]:
print >> finalToc, TOCTEMPLATE.prettify()
print >> finalNcx, NCXTEMPLATE.prettify()

finalToc.close()
finalNcx.close()